# Price action difference

- Analyse differences between CEX and DEX
- In this notebook, we compare Arbitrum `WBTC-USDC` on Uniswap v3 to Binance price action
  of `BTC-USDT`
- We compare one hour candles



# Parameters

Set up parameters we compare.


In [1]:
import pandas as pd
from tradingstrategy.chain import ChainId
from tradingstrategy.timebucket import TimeBucket

time_bucket = TimeBucket.h1
pair_description = (ChainId.arbitrum, "uniswap-v3", "WBTC", "USDC", 0.0005)
binance_data = "../tests/binance-BTCUSDT-1h.parquet"
start = pd.Timestamp("2022-12-21")
end = pd.Timestamp("2023-08-15")

# Load Uniswap on Arbitrum data

In [2]:
from tradingstrategy.timebucket import TimeBucket
from tradingstrategy.pair import PandasPairUniverse, DEXPair
from tradingstrategy.exchange import ExchangeUniverse
from tradingstrategy.client import Client

client = Client.create_jupyter_client()

# Fetch all exchange names, slugs and addresses
exchange_universe: ExchangeUniverse = client.fetch_exchange_universe()

pair_universe = PandasPairUniverse(client.fetch_pair_universe().to_pandas(), exchange_universe)

pair = pair_universe.get_pair_by_human_description(pair_description)

uniswap_candles: pd.DataFrame = client.fetch_candles_by_pair_ids(
    {pair.pair_id},
    TimeBucket.d1,
    progress_bar_description=f"Download data for {pair.get_ticker()}"
)

Started Trading Strategy in Jupyter notebook environment, configuration is stored in /Users/moo/.tradingstrategy


AssertionError: get_pair_by_human_description() needs exchange_universe passed as constructor or function argument in order to do pair lookups

# Load Binance data

In [ ]:
from pathlib import Path
from tradeexecutor.strategy.pandas_trader.alternative_market_data import load_candles_from_parquet

binance_candles, _, binance_time_bucket, _ = load_candles_from_parquet(
    Path("../tests/binance-BTCUSDT-1h.parquet"),
)

# Check data

Make sure both dataframes are in a format we expect.

In [ ]:
from tradingstrategy.charting.candle_chart import validate_ohclv_dataframe

validate_ohclv_dataframe(uniswap_candles)
validate_ohclv_dataframe(binance_candles)

assert binance_time_bucket == time_bucket, "Loaded candles are for different time frame"

# Align data

Make sure both price feeds are aligned to the same time period.


In [ ]:
uniswap_candles = uniswap_candles[start:end]
binance_candles = binance_candles[start:end]

print(f"Time period {start} - {end}")
print(f"Uniswap has {len(uniswap_candles)} candles ")
print(f"Binance has {len(binance_candles)} candles ")
